In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
import dask.dataframe as dd
import dask.distributed as d_dist
from dask.dataframe.core import repartition
from dask.distributed import Client

In [3]:
from importlib import import_module
from importlib.util import find_spec

In [4]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [48]:
%reload_kedro
catalog = context.catalog

2021-08-26 12:36:25,659 - root - INFO - Registered hooks from 2 installed plugin(s): kedro-mlflow-0.7.3
2021-08-26 12:36:25,772 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2021-08-26 12:36:26,132 - root - INFO - ** Kedro project Spotify Recommendations
2021-08-26 12:36:26,134 - root - INFO - Defined global variable `context`, `session` and `catalog`
2021-08-26 12:36:26,146 - root - INFO - Registered line magic `run_viz`


In [6]:
client = d_dist.client._get_global_client() or Client()

In [7]:
split_mpd_pids_to_tids = catalog.load("split_mpd_pids_to_tids")
selected_mpd_track_features = catalog.load("selected_mpd_track_features").compute().set_index("track_spid")

2021-08-26 12:26:08,166 - kedro.io.data_catalog - INFO - Loading data from `split_mpd_pids_to_tids` (ParquetDataSet)...
2021-08-26 12:26:08,288 - kedro.io.data_catalog - INFO - Loading data from `selected_mpd_track_features` (ParquetDataSet)...


In [8]:
split_mpd_pids_to_tids.head(1)

,pid,track_ids,train,test
0,0,"[[3H1LCvO3fVsK2HPguhbml0, 6d8A5sAx9TfdeseDvfWN...","[3H1LCvO3fVsK2HPguhbml0, 6d8A5sAx9TfdeseDvfWNH...","[4pmc2AxSEq6g7hPVlJCPyP, 6sqNctd7MlJoKDOxPVCAv..."


In [9]:
selected_mpd_track_features.head(1)

,track_danceability,track_energy,track_key,track_loudness,track_mode,track_speechiness,track_acousticness,track_instrumentalness,track_liveness,track_valence,track_tempo
track_spid,,,,,,,,,,,
7daKhXFKpT9Zw95fCNDMxG,0.458,0.409,0,-13.886,1,0.0317,0.733,0.916,0.23,0.249,167.352


In [10]:
playlist = split_mpd_pids_to_tids.head(1)

In [11]:
playlist

,pid,track_ids,train,test
0,0,"[[3H1LCvO3fVsK2HPguhbml0, 6d8A5sAx9TfdeseDvfWN...","[3H1LCvO3fVsK2HPguhbml0, 6d8A5sAx9TfdeseDvfWNH...","[4pmc2AxSEq6g7hPVlJCPyP, 6sqNctd7MlJoKDOxPVCAv..."


In [12]:
playlist_tids = playlist["train"].iloc[0]

In [13]:
playlist_tids

array(['3H1LCvO3fVsK2HPguhbml0', '6d8A5sAx9TfdeseDvfWNHd',
       '6MjljecHzHelUDismyKkba', '12qZHAeOyTf93YAWvGDTat',
       '6GIrIt2M39wEGwjCQjGChX', '7H6ev70Weq6DdpZyyTmUXk',
       '69ghzc538EQSVon2Gm3wrr', '1HwpWwa6bnqqRhK8agG4RS',
       '6I9VzXrHxO9rA9A5euc8Ak', '1b7vg5T9YKR3NNqXfBYRF7',
       '2eJ8ij1T3cNUKiGdcUvKhy', '5lDriBxJd22IhOH9zTcFrV',
       '1lzr43nnXAijIGYnCT8M8H', '0WqIKmW4BTrj3eJFmnCKMv',
       '0XUfyU2QviPAs6bxSpXYG4', '5Q0Nhxo0l2bP3pNjpGJwV1',
       '5y69gQtK33qxb8a24ACkCy', '1JURww012QnWAw0zZXi6Aa',
       '3uoQULcUWfnt6nc6J7Vgai', '0UaMYEvWZi0ZqiDOoHU3YI',
       '20ORwCJusz4KS2PbTPVNKo', '2gam98EZKrF9XuOkU13ApN',
       '4omisSlTk6Dsq2iQD7MA07', '7xYnUQigPoIDAMPVK79NEq',
       '34ceTg8ChN5HjrqiIYCn9Q', '1TfAhjzRBWzYZ8IdUV3igl',
       '5c1sfI6wIQEsSUw0xrkFdl', '3utIAb67sOu0QHxBE88P1M',
       '1AWQoqb9bSvzTjaLralEkT', '4I2GqMe7L2ccMpUbnDzYLH',
       '0uqPG793dkDDN7sCUJJIVC', '7oK9VyNzrYvRFo7nQEYkWN',
       '2jFlMILIQzs7lSFudG9lbo', '2nbClS09zsIAqNkshg6jnp

In [14]:
def get_aggregate_track_features(playlist_tids, mpd_track_features):
    return mpd_track_features[mpd_track_features.index.isin(set(playlist_tids.tolist()))].mean()

In [15]:
agg_playlist = get_aggregate_track_features(playlist_tids, selected_mpd_track_features)

In [16]:
agg_playlist

track_danceability          0.674718
track_energy                0.777590
track_key                   5.179487
track_loudness             -4.996897
track_mode                  0.666667
track_speechiness           0.095374
track_acousticness          0.074940
track_instrumentalness      0.000837
track_liveness              0.187151
track_valence               0.633513
track_tempo               119.808949
dtype: float64

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
agg_playlist.to_numpy().reshape(1, -1)

array([[ 6.74717949e-01,  7.77589744e-01,  5.17948718e+00,
        -4.99689744e+00,  6.66666667e-01,  9.53743590e-02,
         7.49398436e-02,  8.36927949e-04,  1.87151282e-01,
         6.33512821e-01,  1.19808949e+02]])

In [19]:
candidate_df = selected_mpd_track_features[~(selected_mpd_track_features.isin(playlist_tids))]

In [20]:
candidate_df

,track_danceability,track_energy,track_key,track_loudness,track_mode,track_speechiness,track_acousticness,track_instrumentalness,track_liveness,track_valence,track_tempo
track_spid,,,,,,,,,,,
7daKhXFKpT9Zw95fCNDMxG,0.458,0.4090,0,-13.886,1,0.0317,0.733000,0.9160,0.2300,0.249,167.352
3rxTCj1eLJi9BK8ULnVTzQ,0.708,0.9310,7,-2.930,0,0.1030,0.137000,0.0000,0.5520,0.656,108.030
3fmbY2XhpkIGyz2FexYPtC,0.749,0.7370,0,-8.879,1,0.0833,0.079000,0.4200,0.1050,0.594,134.038
1emRhNVWm9gg0iKHzuNXGX,0.790,0.6460,6,-10.956,1,0.0544,0.008740,0.9240,0.1180,0.575,127.902
4Z5ZN04NA27qeBO8RApsaZ,0.600,0.8460,4,-5.288,1,0.0319,0.058700,0.0000,0.3190,0.560,96.003
...,...,...,...,...,...,...,...,...,...,...,...
31zAAYl1b2C8gipN9NZjcx,0.664,0.8620,5,-9.467,0,0.0655,0.000484,0.9130,0.1340,0.584,130.025
7vchm2bYxlTLcqJk8Bk7nj,0.521,0.9210,2,-5.573,0,0.0498,0.046000,0.1800,0.3970,0.496,137.001
5gQncDLoBH0DfddK3yp9vY,0.377,0.0801,0,-19.454,1,0.0316,0.924000,0.0000,0.0684,0.307,96.077


In [21]:
playlist_song_matches = cosine_similarity(
    agg_playlist.to_numpy().reshape(1, -1), 
    candidate_df
)

In [22]:
playlist_test_songs = set(playlist["test"].iloc[0])

In [23]:
playlist_test_songs

{'19Js5ypV6JKn4DMExHQbGc',
 '1Bv0Yl01xBDZD4OQP93fyl',
 '1Y4ZdPOOgCUhBcKZOrUFiS',
 '1kusepF3AacIEtUTYrw4GV',
 '2PpruBYCo4H7WOBJ7Q2EwM',
 '3jkdQNkDTxxXtjSO4l0o1H',
 '4E5P1XyAFtrjpiIxkydly4',
 '4Y45aqo9QMa57rDsAJv40A',
 '4pmc2AxSEq6g7hPVlJCPyP',
 '67T6l4q3zVjC5nZZPXByU8',
 '6GIrIt2M39wEGwjCQjGChX',
 '6sqNctd7MlJoKDOxPVCAvU',
 '7DFnq8FYhHMCylykf6ZCxA'}

In [24]:
n=len(playlist_test_songs)

In [25]:
indices = np.argpartition(playlist_song_matches, -n, axis=1)[:, -n:]

In [26]:
predicted_track_ids = candidate_df.iloc[indices[0]].index

In [27]:
predicted_track_ids

Index(['5zUkkQEYvkAnbPpGLBBDa4', '7ExU7Rm36UHTe1wrGmKoxb',
       '6sZrQUotv2ERetBoycJpVc', '18SRUOdjtpB42IeNIHmVwQ',
       '4kS2PFgTKRd5lnh4QY0GIc', '1T8TMbqVchCQKHhfwGO5kv',
       '1FpZJx6RCojuJlvqQwjPu9', '3uKUXYTlA35aEScZWFtSiu',
       '5Bunj5XOX1ojVpgWMraiBk', '5MPCNny11psx06Btopb7Kr',
       '2IqTUjapeQHpQdbaSkTGNQ', '2vT0ZjqKAaFGSJzrHxDG5W',
       '1s9Os30kdCjnxIbJbx3H0e'],
      dtype='object', name='track_spid')

In [28]:
len(set(predicted_track_ids).intersection(playlist_test_songs))/n

0.0

In [29]:
# split_mpd_pids_to_tids["train_agg"] = split_mpd_pids_to_tids["train"].apply(
#     get_aggregate_track_features, args=(selected_mpd_track_features,), meta=("float"))

In [30]:
# playlist_training = split_mpd_pids_to_tids[["pid", "train_agg", "test"]]

In [31]:
# catalog.save("playlist_training", playlist_training)

In [32]:
needed_mpd_pids_to_tids_cats = split_mpd_pids_to_tids[["train","test"]].compute()

In [33]:
#for now subset for experimentation purposes: using 1/100th
#TODO: use dask over the whole dataset
row_limit = needed_mpd_pids_to_tids_cats.shape[0]//100000
subset_needed_mpd_pids_to_tids_cats = needed_mpd_pids_to_tids_cats.sample(row_limit)

In [34]:
subset_needed_mpd_pids_to_tids_cats.shape[0]

10

In [35]:
# def get_top_n_predictions(:
#         playlist_song_matches = cosine_similarity(
#         agg_playlist.to_numpy().reshape(1, -1), 
#         candidate_df
#     )

In [36]:
# subset_needed_mpd_pids_to_tids_cats["train_vector"] = subset_needed_mpd_pids_to_tids_cats["train"].progress_apply(
#     lambda tids: get_aggregate_track_features(tids, selected_mpd_track_features))

In [37]:
import mlflow

In [38]:
# #DB will be a pandas df indexed by the id col you want to query on
# class CosineModel(mlflow.pyfunc.PythonModel):
    
#     def __init__(self, db):
#         self.db = db
        
#     def predict(self, ids, n=5):
#         from sklearn.metrics.pairwise import cosine_similarity
#         import numpy as np
        
#         db = self.db
#         candidate_df = db[~(db.isin(set(ids)))]
#         aggregate_query_feats = self.aggregate(ids)
        
#         similarities = cosine_similarity(
#             aggregate_query_feats.to_numpy().reshape(1, -1), 
#             candidate_df
#         )
#         top_n_indices = np.argpartition(similarities, -n, axis=1)[:, -n:]
#         top_n_predictions = candidate_df.iloc[top_n_indices[0]].index.tolist()
        
#         return top_n_predictions
        
#     def aggregate(self, ids):
#         db = self.db
#         return db[db.index.isin(set(ids))].mean()
    
#     def evaluate(self, ids, test_ids):
#         n = len(test_ids)
#         top_n_predictions = self.predict(ids, n)
#         accuracy = len(set(top_n_predictions).intersection(test_ids)) / n
#         return accuracy

In [39]:
import sys
sys.path.append("../")

In [40]:
from spotify_recommendations.extras.mlflow.models import CosineModel

In [41]:
cosine_model = CosineModel(selected_mpd_track_features)

In [42]:
%%time
cosine_model.predict(playlist_tids)

CPU times: user 4.07 s, sys: 1.61 s, total: 5.68 s
Wall time: 5.67 s


['18SRUOdjtpB42IeNIHmVwQ',
 '1FpZJx6RCojuJlvqQwjPu9',
 '2IqTUjapeQHpQdbaSkTGNQ',
 '2vT0ZjqKAaFGSJzrHxDG5W',
 '1s9Os30kdCjnxIbJbx3H0e']

In [43]:
%%time
cosine_model.evaluate(playlist_tids, playlist_test_songs)

CPU times: user 3.53 s, sys: 750 ms, total: 4.28 s
Wall time: 3.81 s


0.0

In [44]:
all_scores = subset_needed_mpd_pids_to_tids_cats.progress_apply(lambda row: cosine_model.evaluate(row["train"], row["test"]), axis=1)

  0%|          | 0/10 [00:00<?, ?it/s]

In [50]:
catalog.save("cosine_model", cosine_model)

2021-08-26 12:36:42,034 - kedro.io.data_catalog - INFO - Saving data to `cosine_model` (PickleDataSet)...


DataSetError: <class 'spotify_recommendations.extras.mlflow.models.CosineModel'> was not serialized due to: Can't pickle <class 'spotify_recommendations.extras.mlflow.models.CosineModel'>: it's not the same object as spotify_recommendations.extras.mlflow.models.CosineModel